## 0. Install the program
In Bash as below (make sure you execute those steps in the directory holding the ``environment.yml``)

In [ ]:
!cd %PATH_TO_MIIC3_SOURCE_CODE
!conda env create -f environment.yml
!conda activate miic3
!pip install -e .  # Installs to current directory (code changes will be applied to installation right away)

## 1. Preprocess the raw data
All the subfolders are located in ``root``. The hdf5 files holding the preprocessed data will be saved in ``outfolder``
Rawdata needs to be located inside of a folder called ``mseed`` (see *miic3.trace_data.waveform.SDS_FMTSTR* for file system structure)

In [ ]:
from miic3.trace_data.preprocess import Preprocessor
from miic3.trace_data.waveform import Store_Client
from obspy import UTCDateTime
from obspy.clients.fdsn import Client
import os
import time

gfz = Client('GFZ')
root = '/path/to/project'
sc = Store_Client(gfz,root,read_only=False)
# The remove response flag decides whether the Instrument Response is to be removed or not
c = Preprocessor(sc, sampling_rate=25, outfolder='mypreprocessed_data', remove_response=True)
# Start the actual preprocessing, this accepts wildcards
c.preprocess_bulk(network='D0', statlist=['BZG', 'ESO', 'KBG','KIR'], backend='joblib', n_cpus=8)

## 2. Compute Correlations
Once the data is preprocessed, we can compute the actual correlations. All parameters for the correlations are set in ``params.yaml``. MOst importantly, the directories at the beginning of the file need to be run.

Now, we can initiate the correlations using a script containing the code below:

In [ ]:
import yaml
import os
from time import time

with open('/path/to/my/params.yaml') as file:
    di = yaml.load(file, Loader=yaml.FullLoader)

from miic3.correlate.correlate import Correlator
c = Correlator(options=di)
print('correlator initiated')
x = time()
st = c.pxcorr()
print(time()-x)

This is mpi compatible. So we can do the following:

In [ ]:
!mpirun -n %N_CORES python %PATH_TO_THE_SCRIPT_ABOVE

## 3. Plot the results
We access the correlations using the ``CorrelationDataBase`` object

In [ ]:
from miic3.db.corr_hdf5 import CorrelationDataBase
with CorrelationDataBase('/path/to/my/correlations/D0-D0.BZG-ESO.h5') as cdb:
    cst = cdb.get_data('D0-D0', 'BZG-ESO',channel='HHZ-HHZ', tag='subdivision')
# From here on use matplotlib cst works almost like an obspy stream (i.e., it's a list of traces with a data attribute)